# Imports

In [16]:
import pandas as pd

pd.set_option('max_colwidth', 150)

from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from catboost import CatBoostClassifier

In [3]:
DATA_PATH_PREP = '../DATA/prepared'

# Load data

In [4]:
data_table_soft = pd.read_pickle(f'{DATA_PATH_PREP}/04_data_table_soft.pkl')
data_table_med = pd.read_pickle(f'{DATA_PATH_PREP}/04_data_table_med.pkl')
data_table_hard = pd.read_pickle(f'{DATA_PATH_PREP}/04_data_table_hard.pkl')

# Split data

In [7]:
X = data_table_hard.drop('author', axis=1)
y = data_table_hard['author']

# Run a model

In [14]:
strat_k_fold = StratifiedKFold(n_splits=10, shuffle=True)

lr = LogisticRegression()
print(f'LogisticRegression: {cross_val_score(lr, X, y, cv=strat_k_fold).mean()}')

knn = KNeighborsClassifier(n_neighbors = 5)
print(f'KNeighborsClassifier: {cross_val_score(knn, X, y, cv=strat_k_fold).mean()}')

gnb = GaussianNB()
print(f'GaussianNB: {cross_val_score(gnb, X, y, cv=strat_k_fold).mean()}')

mnb = MultinomialNB()
print(f'MultinomialNB: {cross_val_score(mnb, X, y, cv=strat_k_fold).mean()}')

/home/work/Dev/ati/venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


LogisticRegression: 0.625


/home/work/Dev/ati/venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


KNeighborsClassifier: 0.7333333333333333


/home/work/Dev/ati/venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


GaussianNB: 0.7083333333333333


/home/work/Dev/ati/venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


MultinomialNB: 0.5833333333333333


In [21]:
clf = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=15, loss_function='MultiClass')

In [22]:
clf.fit(X, y)

0:	learn: 1.7610786	total: 985ms	remaining: 16m 23s
1:	learn: 1.7193240	total: 1.87s	remaining: 15m 31s
2:	learn: 1.6770860	total: 2.33s	remaining: 12m 56s
3:	learn: 1.6127368	total: 2.61s	remaining: 10m 50s
4:	learn: 1.5792518	total: 3.48s	remaining: 11m 32s
5:	learn: 1.5500063	total: 7.49s	remaining: 20m 41s
6:	learn: 1.5165078	total: 8.03s	remaining: 18m 59s
7:	learn: 1.4787216	total: 37.7s	remaining: 1h 17m 59s
8:	learn: 1.4463853	total: 38.7s	remaining: 1h 11m 5s
9:	learn: 1.4111886	total: 39s	remaining: 1h 4m 16s
10:	learn: 1.3899046	total: 41.2s	remaining: 1h 1m 41s
11:	learn: 1.3460345	total: 41.4s	remaining: 56m 49s
12:	learn: 1.3148172	total: 42.8s	remaining: 54m 6s
13:	learn: 1.2943705	total: 44.9s	remaining: 52m 40s
14:	learn: 1.2631226	total: 45.4s	remaining: 49m 41s
15:	learn: 1.2460106	total: 46.3s	remaining: 47m 28s
16:	learn: 1.2141947	total: 46.8s	remaining: 45m 6s
17:	learn: 1.1825220	total: 47.1s	remaining: 42m 47s
18:	learn: 1.1621562	total: 50.8s	remaining: 43m 41

In [ ]:
from sklearn.model_selection import cross_validate

# Perform cross-validation
scores = cross_validate(clf, X, y, cv=5, scoring='neg_log_loss', return_train_score=True)

# Print the mean and standard deviation of the scores
# print("Mean score: ", np.mean(scores))
# print("Standard deviation: ", np.std(scores))
print(scores)